In [49]:
from JHProgress import JHProgress
import pandas as pd
import numpy as np

In [105]:
jhp=JHProgress()
#jhp.cargar_fundamentales()
#jhp.guardar_fundamentales_sql()
prices=jhp.descargar_precios_yahoo("2023-01-01","2025-12-07")
fundamentales=jhp.cargar_fundamentales_sql()

2025-12-07 16:26:41,341 - INFO - Tickers downloaded successfully
c:\Users\palaj\OneDrive\Escritorio\JHProgress\Dev\JHProgress.py:129: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(self.config['Simbolos'], start=start, end=end, progress=False)
2025-12-07 16:26:42,400 - INFO - DataFrame de precios descargado exitosamente
2025-12-07 16:26:53,270 - INFO - DataFrame de fundamentales cargado desde la tabla VW_FUNDAMENTALES_PROCESADOS exitosamente


In [60]:
ss=df[df['StockholdersEquityNoteStockSplitConversionRatio1']>0].reset_index().groupby('StockholdersEquityNoteStockSplitConversionRatio1',).max()['FECHA_FIN']
ss

StockholdersEquityNoteStockSplitConversionRatio1
4.0     2021-07-19
10.0    2024-06-30
Name: FECHA_FIN, dtype: object

In [ ]:
df=fundamentales.pivot(index=['FECHA_FIN','start','TIPO'],columns=['Symbol','Metrica'],values='VALOR_AJUSTADO')
df=df['NVDA'].dropna(how='all')
df=df.groupby('FECHA_FIN').max()
ss=df[df['StockholdersEquityNoteStockSplitConversionRatio1']>0].reset_index().groupby('StockholdersEquityNoteStockSplitConversionRatio1',).max()['FECHA_FIN']

df_proc=df.copy()
for i in ss.index:
    df_proc.loc[:ss[i],'WeightedAverageNumberOfDilutedSharesOutstanding']*=i

df_proc=df_proc.dropna(subset=['NetIncomeLoss']).drop(columns=['StockholdersEquityNoteStockSplitConversionRatio1'])
df_proc['EarningsPerShareDiluted']=df_proc['NetIncomeLoss']/df_proc['WeightedAverageNumberOfDilutedSharesOutstanding']
df_proc['Equity']=df_proc['Assets']-df_proc['Liabilities']
df_proc['ROE']=df_proc['NetIncomeLoss']/df_proc['Equity']
df_proc['ROA']=df_proc['NetIncomeLoss']/df_proc['Assets']
df_proc['BookValuePerShare']=df_proc['Equity']/df_proc['WeightedAverageNumberOfDilutedSharesOutstanding']
df_proc['SalesPerShare']=df_proc['Revenues']/df_proc['WeightedAverageNumberOfDilutedSharesOutstanding']
df_proc['AssetTurnover']=df_proc['Revenues']/df_proc['Assets']
df_proc['DebtToEquity']=df_proc['Liabilities']/df_proc['Equity']
df_proc.index=df_proc.index.astype('datetime64[ns]')
df_proc=df_proc.reset_index()
df_proc['Fin_Periodo']=df_proc['FECHA_FIN'].shift(-1).fillna(pd.Timestamp('2100-12-31'))
df_proc=df_proc.reset_index().merge(prices['Close']['NVDA'].to_frame().reset_index(),how='cross')
df_proc=df_proc[(df_proc['Date']>=df_proc['FECHA_FIN']) & (df_proc['Date']<=df_proc['Fin_Periodo'])]
df_proc['PE_Ratio']=df_proc['NVDA']/df_proc['EarningsPerShareDiluted']
df_proc['PB_Ratio']=df_proc['NVDA']/df_proc['BookValuePerShare']
df_proc['PS_Ratio']=df_proc['NVDA']/df_proc['SalesPerShare']
df_proc['MarketCap']=df_proc['NVDA']*df_proc['WeightedAverageNumberOfDilutedSharesOutstanding']

df_proc

,index,FECHA_FIN,EarningsPerShareDiluted,GrossProfit,NetIncomeLoss,OperatingIncomeLoss,Revenues,Assets,Liabilities,NoncurrentAssets,...,Equity,ROE,ROA,BookValuePerShare,SalesPerShare,AssetTurnover,DebtToEquity,Fin_Periodo,Date,NVDA
41895,57,2022-10-30,0.027016,3.177000e+09,6.800000e+08,6.010000e+08,5.931000e+09,4.048800e+10,1.913900e+10,NaN,...,2.134900e+10,0.031852,0.016795,0.848192,0.235638,0.146488,0.896482,2023-01-29,2023-01-03,14.300683
41896,57,2022-10-30,0.027016,3.177000e+09,6.800000e+08,6.010000e+08,5.931000e+09,4.048800e+10,1.913900e+10,NaN,...,2.134900e+10,0.031852,0.016795,0.848192,0.235638,0.146488,0.896482,2023-01-29,2023-01-04,14.734249
41897,57,2022-10-30,0.027016,3.177000e+09,6.800000e+08,6.010000e+08,5.931000e+09,4.048800e+10,1.913900e+10,NaN,...,2.134900e+10,0.031852,0.016795,0.848192,0.235638,0.146488,0.896482,2023-01-29,2023-01-05,14.250735
41898,57,2022-10-30,0.027016,3.177000e+09,6.800000e+08,6.010000e+08,5.931000e+09,4.048800e+10,1.913900e+10,NaN,...,2.134900e+10,0.031852,0.016795,0.848192,0.235638,0.146488,0.896482,2023-01-29,2023-01-06,14.844141
41899,57,2022-10-30,0.027016,3.177000e+09,6.800000e+08,6.010000e+08,5.931000e+09,4.048800e+10,1.913900e+10,NaN,...,2.134900e+10,0.031852,0.016795,0.848192,0.235638,0.146488,0.896482,2023-01-29,2023-01-09,15.612370
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51445,69,2025-10-26,1.300220,4.184900e+10,3.191000e+10,3.601000e+10,5.700600e+10,1.611480e+11,4.225100e+10,NaN,...,1.188970e+11,0.268384,0.198017,4.844634,2.322794,0.353749,0.355358,2100-12-31,2025-12-01,179.909988
51446,69,2025-10-26,1.300220,4.184900e+10,3.191000e+10,3.601000e+10,5.700600e+10,1.611480e+11,4.225100e+10,NaN,...,1.188970e+11,0.268384,0.198017,4.844634,2.322794,0.353749,0.355358,2100-12-31,2025-12-02,181.449905
51447,69,2025-10-26,1.300220,4.184900e+10,3.191000e+10,3.601000e+10,5.700600e+10,1.611480e+11,4.225100e+10,NaN,...,1.188970e+11,0.268384,0.198017,4.844634,2.322794,0.353749,0.355358,2100-12-31,2025-12-03,179.580002
51448,69,2025-10-26,1.300220,4.184900e+10,3.191000e+10,3.601000e+10,5.700600e+10,1.611480e+11,4.225100e+10,NaN,...,1.188970e+11,0.268384,0.198017,4.844634,2.322794,0.353749,0.355358,2100-12-31,2025-12-04,183.380005


,index,FECHA_FIN,EarningsPerShareDiluted,GrossProfit,NetIncomeLoss,OperatingIncomeLoss,Revenues,Assets,Liabilities,NoncurrentAssets,...,Equity,ROE,ROA,BookValuePerShare,SalesPerShare,AssetTurnover,DebtToEquity,Fin_Periodo,Date,NVDA
0,0,2008-07-27,-0.005105,1.499170e+08,-1.209290e+08,-1.553920e+08,8.926760e+08,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.037686,NaN,NaN,2008-10-26,2023-01-03,14.300683
1,0,2008-07-27,-0.005105,1.499170e+08,-1.209290e+08,-1.553920e+08,8.926760e+08,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.037686,NaN,NaN,2008-10-26,2023-01-04,14.734249
2,0,2008-07-27,-0.005105,1.499170e+08,-1.209290e+08,-1.553920e+08,8.926760e+08,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.037686,NaN,NaN,2008-10-26,2023-01-05,14.250735
3,0,2008-07-27,-0.005105,1.499170e+08,-1.209290e+08,-1.553920e+08,8.926760e+08,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.037686,NaN,NaN,2008-10-26,2023-01-06,14.844141
4,0,2008-07-27,-0.005105,1.499170e+08,-1.209290e+08,-1.553920e+08,8.926760e+08,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.037686,NaN,NaN,2008-10-26,2023-01-09,15.612370
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51445,69,2025-10-26,1.300220,4.184900e+10,3.191000e+10,3.601000e+10,5.700600e+10,1.611480e+11,4.225100e+10,NaN,...,1.188970e+11,0.268384,0.198017,4.844634,2.322794,0.353749,0.355358,2100-12-31,2025-12-01,179.909988
51446,69,2025-10-26,1.300220,4.184900e+10,3.191000e+10,3.601000e+10,5.700600e+10,1.611480e+11,4.225100e+10,NaN,...,1.188970e+11,0.268384,0.198017,4.844634,2.322794,0.353749,0.355358,2100-12-31,2025-12-02,181.449905
51447,69,2025-10-26,1.300220,4.184900e+10,3.191000e+10,3.601000e+10,5.700600e+10,1.611480e+11,4.225100e+10,NaN,...,1.188970e+11,0.268384,0.198017,4.844634,2.322794,0.353749,0.355358,2100-12-31,2025-12-03,179.580002
51448,69,2025-10-26,1.300220,4.184900e+10,3.191000e+10,3.601000e+10,5.700600e+10,1.611480e+11,4.225100e+10,NaN,...,1.188970e+11,0.268384,0.198017,4.844634,2.322794,0.353749,0.355358,2100-12-31,2025-12-04,183.380005


,index,FECHA_FIN,EarningsPerShareDiluted,GrossProfit,NetIncomeLoss,OperatingIncomeLoss,Revenues,Assets,Liabilities,NoncurrentAssets,...,Equity,ROE,ROA,BookValuePerShare,SalesPerShare,AssetTurnover,DebtToEquity,Fin_Periodo,Date,NVDA
41895,57,2022-10-30,0.027016,3.177000e+09,6.800000e+08,6.010000e+08,5.931000e+09,4.048800e+10,1.913900e+10,NaN,...,2.134900e+10,0.031852,0.016795,0.848192,0.235638,0.146488,0.896482,2023-01-29,2023-01-03,14.300683
41896,57,2022-10-30,0.027016,3.177000e+09,6.800000e+08,6.010000e+08,5.931000e+09,4.048800e+10,1.913900e+10,NaN,...,2.134900e+10,0.031852,0.016795,0.848192,0.235638,0.146488,0.896482,2023-01-29,2023-01-04,14.734249
41897,57,2022-10-30,0.027016,3.177000e+09,6.800000e+08,6.010000e+08,5.931000e+09,4.048800e+10,1.913900e+10,NaN,...,2.134900e+10,0.031852,0.016795,0.848192,0.235638,0.146488,0.896482,2023-01-29,2023-01-05,14.250735
41898,57,2022-10-30,0.027016,3.177000e+09,6.800000e+08,6.010000e+08,5.931000e+09,4.048800e+10,1.913900e+10,NaN,...,2.134900e+10,0.031852,0.016795,0.848192,0.235638,0.146488,0.896482,2023-01-29,2023-01-06,14.844141
41899,57,2022-10-30,0.027016,3.177000e+09,6.800000e+08,6.010000e+08,5.931000e+09,4.048800e+10,1.913900e+10,NaN,...,2.134900e+10,0.031852,0.016795,0.848192,0.235638,0.146488,0.896482,2023-01-29,2023-01-09,15.612370
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51445,69,2025-10-26,1.300220,4.184900e+10,3.191000e+10,3.601000e+10,5.700600e+10,1.611480e+11,4.225100e+10,NaN,...,1.188970e+11,0.268384,0.198017,4.844634,2.322794,0.353749,0.355358,2100-12-31,2025-12-01,179.909988
51446,69,2025-10-26,1.300220,4.184900e+10,3.191000e+10,3.601000e+10,5.700600e+10,1.611480e+11,4.225100e+10,NaN,...,1.188970e+11,0.268384,0.198017,4.844634,2.322794,0.353749,0.355358,2100-12-31,2025-12-02,181.449905
51447,69,2025-10-26,1.300220,4.184900e+10,3.191000e+10,3.601000e+10,5.700600e+10,1.611480e+11,4.225100e+10,NaN,...,1.188970e+11,0.268384,0.198017,4.844634,2.322794,0.353749,0.355358,2100-12-31,2025-12-03,179.580002
51448,69,2025-10-26,1.300220,4.184900e+10,3.191000e+10,3.601000e+10,5.700600e+10,1.611480e+11,4.225100e+10,NaN,...,1.188970e+11,0.268384,0.198017,4.844634,2.322794,0.353749,0.355358,2100-12-31,2025-12-04,183.380005


In [110]:
df_proc.index

DatetimeIndex(['2008-07-27', '2008-10-26', '2009-01-25', '2009-04-26',
               '2009-07-26', '2009-10-25', '2010-01-31', '2010-05-02',
               '2010-08-01', '2010-10-31', '2011-01-30', '2011-05-01',
               '2011-07-31', '2011-10-30', '2012-01-29', '2012-04-29',
               '2012-07-29', '2012-10-28', '2013-01-27', '2013-04-28',
               '2013-07-28', '2013-10-27', '2014-01-26', '2014-04-27',
               '2014-07-27', '2014-10-26', '2015-01-25', '2015-04-26',
               '2015-07-26', '2015-10-25', '2016-01-31', '2016-05-01',
               '2016-07-31', '2016-10-30', '2017-01-29', '2017-04-30',
               '2017-07-30', '2017-10-29', '2018-01-28', '2018-04-29',
               '2018-07-29', '2018-10-28', '2019-01-27', '2019-04-28',
               '2019-07-28', '2019-10-27', '2020-01-26', '2020-04-26',
               '2020-07-26', '2020-10-25', '2021-01-31', '2021-05-02',
               '2021-08-01', '2021-10-31', '2022-01-30', '2022-05-01',
      

In [111]:
prices.index

DatetimeIndex(['2023-01-03', '2023-01-04', '2023-01-05', '2023-01-06',
               '2023-01-09', '2023-01-10', '2023-01-11', '2023-01-12',
               '2023-01-13', '2023-01-17',
               ...
               '2025-11-21', '2025-11-24', '2025-11-25', '2025-11-26',
               '2025-11-28', '2025-12-01', '2025-12-02', '2025-12-03',
               '2025-12-04', '2025-12-05'],
              dtype='datetime64[ns]', name='Date', length=735, freq=None)